In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50,InceptionResNetV2
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras import optimizers
import matplotlib.pyplot as plt

In [ ]:
base_model = InceptionResNetV2(weights = 'imagenet',include_top=False,input_shape=(224,224,3))

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
for layer in base_model.layers: 
  layer.trainable = False
model.compile(optimizer=optimizers.SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.load_weights("newrust.h5")